In [1]:
# Outside imports
import os
import importlib
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import tensorflow as tf
import tensorflow.keras.backend as K

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [58]:

import train
import evaluate
import data_util.data
import data_util.batcher
import data_util.config


importlib.reload(train)
importlib.reload(evaluate)
importlib.reload(data_util.config)
importlib.reload(data_util.data)
importlib.reload(data_util.batcher)

from train import *
from evaluate import *
from data_util.data import *
from data_util.batcher import *


INFO:tensorflow:Bucket queue size: 1000, Input queue size: 10000


In [7]:
# load real data
data_path = '/Users/rowancassius/Desktop/capstone/data/context_task_trim.tsv'
dat = pd.read_csv(data_path, sep='\t')

# test/train split
x_train, x_test, y_train, y_test = train_test_split(
    dat.TaskSentence.values, 
    dat.Summary.values,
    test_size=0.2,
    random_state=0,
    shuffle=True
)

# fit tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(list(x_train)+list(y_train))

examples = list(zip(x_train, y_train))
# examples

In [46]:
# Make Vocab
vocab = data.Vocab(
    words = tokenizer.word_index.keys(), 
    max_size=len(tokenizer.word_index))

# Make Batcher
batcher = Batcher(
    examples=examples,
    vocab=vocab, 
    mode='train', 
    batch_size=10, 
    single_pass=False)

max_size of vocab was specified as 1546; we now have 1546 words. Stopping reading.
Finished constructing vocabulary of 1546 total words. Last word added: loi
INFO:tensorflow:Bucket queue size: 8, Input queue size: 428


In [47]:
random.seed(123)
T.manual_seed(123)
if T.cuda.is_available():
    T.cuda.manual_seed_all(123)

In [48]:
class Namespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

In [49]:
train_mle = "yes"
train_rl = "no"
mle_weight = 1.0
load_model = None
new_lr = None
rl_weight = 1 - mle_weight

opt = Namespace(train_mle = train_mle, 
                train_rl = train_rl, 
                mle_weight = mle_weight, 
                load_model = load_model,
                new_lr = new_lr, 
                rl_weight = rl_weight)


train_processor = Train(vocab, batcher, opt)

In [50]:
train_processor.trainIters()

0
1
2
3
4
iter: 5 mle_loss: 8.556 reward: 0.0000
5
6
7
8
9
iter: 10 mle_loss: 6.686 reward: 0.0000
model saved at: 
 data/saved_models/0000010.tar
10
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 10000


In [59]:
task = "validate"
load_model = os.path.join(config.log_root, "data/saved_models/0000010.tar")

opt = Namespace(task = task, load_model = load_model)

# new batcher for evaluation
batcher = Batcher(
    examples=examples[:20],
    vocab=vocab, 
    mode='train', 
    batch_size=10, 
    single_pass=True)

eval_processor = Evaluate(vocab, batcher, opt) 

example_generator completed reading all examples. No more data.


Exception in thread Thread-28:
Traceback (most recent call last):
  File "/Users/rowancassius/Desktop/capstone/LSTM_Summarizer/data_util/batcher.py", line 273, in text_generator
    e = next(example_generator) # e is a tf.Example
StopIteration

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/rowancassius/opt/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/Users/rowancassius/opt/anaconda3/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/rowancassius/Desktop/capstone/LSTM_Summarizer/data_util/batcher.py", line 207, in fill_example_queue
    (article, abstract) = next(input_gen) # read the next example from file. article and abstract are both strings.
RuntimeError: generator raised StopIteration



In [60]:
eval_processor.evaluate_batch(print_sents = True)

Summarizing Batch...
Summarizing Batch...
INFO:tensorflow:Finished reading dataset in single_pass mode.


(['xxx',
  'xxx',
  'xxx',
  'xxx',
  'xxx',
  'xxx',
  'xxx',
  'xxx',
  'Please Chris Germany@enron.com Germany@enron.com',
  'Can look? look? look?',
  'xxx',
  'xxx',
  'xxx',
  'xxx',
  'xxx',
  'xxx',
  'Please Ellen.',
  'Can tomorrow? tomorrow?',
  'Give call. call. call.',
  'Please scroll down down down'],
 ['Open ECT Address Book',
  'Attend meeting about responsibilities in maintaining the Kerr McGee tickets in Sitara',
  "Insert appropriate Seller's Payment",
  'Review Alberta PPA presentation',
  'Review filing with the FERC and send coments to Mollie Lampi',
  'Complete survey',
  'Put SENDER on cc list for Wellesley',
  'Examine this and call SENDER',
  'Reply to Chris at Germany@enron.com',
  'Look at definition for diesel index',
  'Donate this money',
  'Create swaps today',
  'Supply Portland at their system',
  'Call SENDER',
  'Check work phone for Joseph Brophy',
  'Call SENDER or Tracy Cummins at 713-8566-525',
  'Run minibook and forward to Ellen',
  'Meet with

INFO:tensorflow:Bucket queue size: 1000, Input queue size: 10000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 10000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 10000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 10000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 10000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 10000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 10000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 10000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 10000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 10000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 10000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 10000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 10000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 10000
INFO:tensorflow:Bucket queue size: 1000, Input queue size: 10000
INFO:tensorflow:Bucket qu